<a href="https://colab.research.google.com/github/BhavaniAM/SRVP/blob/master/Video_Prediction_SRVP_MMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive
---

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/Colab/Notebooks/Video_Prediction/SRVP/MMNIST

/gdrive/MyDrive/Colab/Notebooks/VideoPrediction/SRVP/MMNIST


Install dependencies
---

In [ ]:
# ! mkdir det_test_set
! python3 -m pip install configargparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Create the testset
---

Generate the testset and run the testing code on the dataset (for total 25 frames)
---

In [ ]:
!python -m preprocessing.mmnist.make_test_set --data_dir datasets/work_det_mmnist --deterministic --seq_len 25

9913344it [00:00, 38966777.99it/s]                 
Extracting datasets/work_det_mmnist/MNIST/raw/train-images-idx3-ubyte.gz to datasets/work_det_mmnist/MNIST/raw

29696it [00:00, 48483476.68it/s]
Extracting datasets/work_det_mmnist/MNIST/raw/train-labels-idx1-ubyte.gz to datasets/work_det_mmnist/MNIST/raw

1649664it [00:00, 11014665.09it/s]               
Extracting datasets/work_det_mmnist/MNIST/raw/t10k-images-idx3-ubyte.gz to datasets/work_det_mmnist/MNIST/raw

5120it [00:00, 17332394.25it/s]
Extracting datasets/work_det_mmnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to datasets/work_det_mmnist/MNIST/raw

100% 16/16 [00:00<00:00, 512.11it/s]
Saving testset at datasets/work_det_mmnist/mmnist_test_2digits_64.npz


In [ ]:
! mkdir '/content/predicted'
! mkdir '/content/target'
! mkdir '/content/conditional'

In [ ]:
! pip install 'tensorflow>=1.15, <2'
! pip install tensorflow-gan==1.0.0.dev0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 1.1 kB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 3.8 MB 58.7 MB/s 
     |████████████████████████████████| 503 kB 51.6 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 2.9 MB 47.3 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=5af383f64f759bbef095d52bb94da6acbdf831bccaa12a935ee32d3a3e55335e
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninst

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 317 kB 21.6 MB/s 


In [ ]:
!python test.py --data_dir datasets/work_det_mmnist --xp_dir pretrained_models/mmnist/deterministic --lpips_dir weights --n_samples 1 --nt_gen 25 --fvd



Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100% 233M/233M [00:01<00:00, 158MB/s]
nt_cond line 178 =  5
nt_test (total_frames) line 180 =  25
Loading data...
Loading model...
Evaluation...
100% 1/1 [00:06<00:00,  6.41s/it]
Computing FVD...

2022-06-29 12:34:40.981191: W tensorflow/core/graph/graph_constructor.cc:1502] Importing a graph with a lower producer version 26 into an existing graph with producer version 134. Shape inference will have run different parts of the graph with different producer versions.
2022-06-29 12:34:43.113009: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-29 12:34:43.116728: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200195000 Hz
2022-06-29 12:34:43.116944: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x943f9c0 

Generate metrics.csv
---

In [ ]:
from zipfile import ZipFile
with ZipFile('pretrained_models/mmnist/full_results_mmnist_srvp.zip', 'r') as z:
  z.extractall('pretrained_models/mmnist/')

In [ ]:
!pwd

/gdrive/MyDrive/Colab/Notebooks/VideoPrediction/SRVP/MMNIST


In [ ]:
import numpy as np
full_results = np.load('pretrained_models/mmnist/full_results_mmnist_det_srvp/results.npz')

In [ ]:
import pandas as pd

ssim_best = full_results['ssim']
psnr_best = full_results['psnr']
lpips_best = full_results['lpips']

In [ ]:
num_vid = [i for i in range(1,5001)]
dict = {'num_vid': num_vid, 'ssim_best': ssim_best,'psnr_best': psnr_best,'lpips_best': lpips_best} 
df = pd.DataFrame(dict) 
df.to_csv('full_metrics_srvp.csv', index = False)

In [ ]:
import numpy as np
results_best = np.load('pretrained_models/mmnist/deterministic/results.npz')
results_worst = np.load('pretrained_models/mmnist/deterministic/results_worst.npz')

In [ ]:
import pandas as pd

ssim_best = results_best['ssim']
psnr_best = results_best['psnr']
lpips_best = results_best['lpips']

ssim_worst = results_worst['ssim']
psnr_worst = results_worst['psnr']
lpips_worst = results_worst['lpips']

print(ssim_best)
print(ssim_worst)

[0.93404293 0.8718611  0.84540784 0.97903603 0.8085613  0.8566801
 0.8569953  0.93589735 0.9255928  0.8572391  0.93022317 0.9215609
 0.95586604 0.93568116 0.93319017 0.88424224]
[0.93404293 0.8718611  0.84540784 0.97903603 0.8085613  0.8566801
 0.8569953  0.93589735 0.9255928  0.8572391  0.93022317 0.9215609
 0.95586604 0.93568116 0.93319017 0.88424224]


In [ ]:
# num_vid = [i for i in range(1,opt.batch_size+1)]
num_vid = [i for i in range(1,16+1)]
dict = {'num_vid': num_vid, 'ssim_best': ssim_best, 'ssim_worst': ssim_worst, 'psnr_best': psnr_best, 'psnr_worst': psnr_worst, 'lpips_best': lpips_best, 'lpips_worst': lpips_worst} 
df = pd.DataFrame(dict) 
df.to_csv('metrics.csv', index = False)

# Best and worst frames from the full metrics

In [ ]:
import numpy as np
import torch
from google.colab import files
import matplotlib.pyplot as plt

In [ ]:
# To remove a directory (can be empty or can have data)
%rm -rf /content/ssim_best
%rm -rf /content/ssim_worst
%rm -rf /content/psnr_best
%rm -rf /content/psnr_worst

In [ ]:
import pandas as pd
df = pd.read_csv('./full_metrics_srvp.csv')

In [ ]:
df

,num_vid,ssim_best,psnr_best,lpips_best
0,1,0.981451,28.268100,0.011920
1,2,0.928169,22.409868,0.058840
2,3,0.963404,25.608347,0.023195
3,4,0.979957,27.944218,0.016162
4,5,0.903375,21.368279,0.070052
...,...,...,...,...
4995,4996,0.972117,26.557438,0.010915
4996,4997,0.969855,25.056232,0.031366
4997,4998,0.893049,21.270290,0.055799
4998,4999,0.954299,24.131964,0.048295


In [ ]:
df.sort_values('ssim_best',inplace = True)
df

,num_vid,ssim_best,psnr_best,lpips_best
931,932,0.637157,12.716866,0.184999
3173,3174,0.699888,13.601788,0.096040
3373,3374,0.711808,15.930035,0.235957
1239,1240,0.717641,15.389223,0.161445
1815,1816,0.742096,16.546064,0.124158
...,...,...,...,...
401,402,0.989375,31.127708,0.012440
123,124,0.989597,29.636803,0.007844
3283,3284,0.989645,31.765985,0.007665
4618,4619,0.990070,31.651264,0.015941


In [ ]:
df.sort_values('psnr_best',inplace = True)
df

,num_vid,ssim_best,psnr_best,lpips_best
931,932,0.637157,12.716866,0.184999
3173,3174,0.699888,13.601788,0.096040
310,311,0.743041,14.960924,0.148059
1239,1240,0.717641,15.389223,0.161445
909,910,0.763544,15.470829,0.111363
...,...,...,...,...
401,402,0.989375,31.127708,0.012440
4717,4718,0.987603,31.200440,0.014395
594,595,0.987877,31.298973,0.012177
4618,4619,0.990070,31.651264,0.015941


In [ ]:
df.sort_values('lpips_best',inplace = True)
df

,num_vid,ssim_best,psnr_best,lpips_best
4932,4933,0.988959,30.022892,0.005228
4246,4247,0.982476,27.645227,0.005413
4371,4372,0.982145,27.381702,0.007112
2926,2927,0.982266,27.684483,0.007368
2816,2817,0.984798,28.749820,0.007419
...,...,...,...,...
931,932,0.637157,12.716866,0.184999
4084,4085,0.779402,18.398190,0.187316
168,169,0.847337,18.647545,0.187370
257,258,0.774736,16.810200,0.207712


In [ ]:
full_results = np.load('pretrained_models/mmnist/full_results_mmnist_det_srvp/ssim_best.npz')
print(full_results['samples'].shape)

(5000, 20, 64, 64, 1)


In [ ]:
!mkdir '/content/ssim_best'
full_results = np.load('pretrained_models/mmnist/full_results_mmnist_det_srvp/ssim_best.npz')
new_video = full_results['samples'].reshape(5000, 20, 1, 64, 64)
num_frames = 20
batch = 5000
new_save_number = 1
for i in range(batch):
  if(i == 4618 or i == 3349):
    for j in range(num_frames):
      plt.imsave('/content/ssim_best/x_ssim_best_' + str(i) + "_" + str(new_save_number) + '.png', new_video[i][j].reshape(64,64), cmap='gray')
      new_save_number = new_save_number + 1

In [ ]:
!mkdir /content/ssim_worst
full_results = np.load('pretrained_models/mmnist/full_results_mmnist_det_srvp/ssim_worst.npz')
new_video = full_results['samples'].reshape(5000, 20, 1, 64, 64)
num_frames = 20
batch = 5000
new_save_number = 1
for i in range(batch):
  if(i == 931 or i == 3173):
    for j in range(num_frames):
      plt.imsave('/content/ssim_worst/x_ssim_worst_' + str(i) + "_" + str(new_save_number) + '.png', new_video[i][j].reshape(64,64), cmap='gray')
      new_save_number = new_save_number + 1

In [ ]:
%rm -rf /content/psnr_best
!mkdir '/content/psnr_best'
full_results = np.load('pretrained_models/mmnist/full_results_mmnist_det_srvp/psnr_best.npz')
new_video = full_results['samples'].reshape(5000, 20, 1, 64, 64)
num_frames = 20
batch = 5000
new_save_number = 1
for i in range(batch):
  if(i == 4618 or i == 3283):
    for j in range(num_frames):
      plt.imsave('/content/psnr_best/x_psnr_best_' + str(i) + "_" + str(new_save_number) + '.png', new_video[i][j].reshape(64,64), cmap='gray')
      new_save_number = new_save_number + 1

In [ ]:
!mkdir /content/psnr_worst
full_results = np.load('pretrained_models/mmnist/full_results_mmnist_det_srvp/psnr_worst.npz')
new_video = full_results['samples'].reshape(5000, 20, 1, 64, 64)
num_frames = 20
batch = 5000
new_save_number = 1
for i in range(batch):
  if(i == 931 or i == 3173):
    for j in range(num_frames):
      plt.imsave('/content/psnr_worst/x_psnr_worst' + str(i) + "_" + str(new_save_number) + '.png', new_video[i][j].reshape(64,64), cmap='gray')
      new_save_number = new_save_number + 1

In [ ]:
%rm -rf /content/lpips_best
!mkdir '/content/lpips_best'
full_results = np.load('pretrained_models/mmnist/full_results_mmnist_det_srvp/lpips_best.npz')
new_video = full_results['samples'].reshape(5000, 20, 1, 64, 64)
num_frames = 20
batch = 5000
new_save_number = 1
for i in range(batch):
  if(i == 4932 or i == 4246):
    for j in range(num_frames):
      plt.imsave('/content/lpips_best/x_lpips_best_' + str(i) + "_" + str(new_save_number) + '.png', new_video[i][j].reshape(64,64), cmap='gray')
      new_save_number = new_save_number + 1

In [ ]:
%rm -rf /content/lpips_worst
!mkdir /content/lpips_worst
full_results = np.load('pretrained_models/mmnist/full_results_mmnist_det_srvp/lpips_worst.npz')
new_video = full_results['samples'].reshape(5000, 20, 1, 64, 64)
num_frames = 20
batch = 5000
new_save_number = 1
for i in range(batch):
  if(i == 257 or i == 3373):
    for j in range(num_frames):
      plt.imsave('/content/lpips_worst/x_lpips_worst' + str(bhav) + "_" + str(new_save_number) + '.png', new_video[i][j].reshape(64,64), cmap='gray')
      new_save_number = new_save_number + 1